# Algoritmi MCTS. Aplicație: Connect Four
 - Tudor Berariu
 - Andrei Olaru

## Scopul laboratorului

Scopul acestui laborator este acela de a implementa un algoritm din familia **MCTS** (_Monte Carlo Tree Search_), și anume **UCT** (_Upper Confidence Bound for Trees_).

Pentru a testa algoritmul vom folosi jocul _Connect Four_.

Prima parte a laboratorului construiește funcțiile necesare desfășurarea jocului _Connect Four_, iar cea de-a doua parte urmărește implementarea pas cu pas a algoritmului **UCT**.

#### Resurse

https://en.wikipedia.org/wiki/Monte_Carlo_tree_search

Slides curs IA despre Căutare în Jocuri

## Jocul _Connect Four_

### Descrierea jocului

Jocul _Connect Four_ [(link)](https://en.wikipedia.org/wiki/Connect_Four) lucrează cu o matrice verticală de **înălțime 6** și **lățime 7** în care doi jucători dau drumul unor jetoane de culori diferite (un jucător are jetoane roșii, iar celălalt albastre). La fiecare mutare, un jucător alege o coloană din cele 7 și dă drumul jetonului. Acesta _cade_, așezându-se pe prima poziție liberă din acea coloană. Într-o coloană nu se pot așeza mai mult de șase jetoane. Câștigă acel jucător care reușește să așeze *patru* dintre jetoanele lui (de aceeași culoare) într-o linie pe orizontală, verticală sau diagonală.

### Reprezentarea stărilor

Starea jocului va fi reprezentată printr-un tuplu format din două elemente:
 - o listă ce va conține 7 liste corespunzătoare celor 7 coloane
   + fiecare listă va avea lungimea 6 și va conține 0 (poziție liberă), 1 (jeton roșu) și 2 (jeton albastru)
   + poziția 0 din listă corespunde rândului cel mai de jos
 - indicatorul jucătorului ce trebuie să _mute_: 1 pentru roșu, 2 pentru jucătorul albastru.

In [32]:
# Dimensiunile matricei
HEIGHT, WIDTH = 6, 7

# Pozițiile din tuplul ce constituie o stare
BOARD, NEXT_PLAYER = 0, 1

# Jucătorii
RED, BLUE, DRAW = 1, 2, 3
name = ["", "ROȘU", "ALBASTRU", "REMIZĂ"]

# Funcție ce întoarce o stare inițială
def init_state():
    return ([[0 for row in range(HEIGHT)] for col in range(WIDTH)], RED)

# Funcție ce afișează o stare
def print_state(state):
    for row in range(HEIGHT-1, -1, -1):
        ch = " RA"
        l = map(lambda col: ch[state[BOARD][col][row]], range(WIDTH))
        print("|" + "".join(l) + "|")
    print("+" + "".join("-" * WIDTH) + "+")
    print("Urmează: %d - %s" % (state[NEXT_PLAYER], name[state[NEXT_PLAYER]]))

# Se afișează starea inițială a jocului
print("Starea inițială:")
print_state(init_state())

Starea inițială:
|       |
|       |
|       |
|       |
|       |
|       |
+-------+
Urmează: 1 - ROȘU


### Mutările

Cum toată informația necesară pentru a descrie o mutare este dată de coloana în care un jucător a ales să își așeze jetonul, o acțiune va fi reprezentată simplu printr-un număr.

**Cerința 1:** Completați funcția `get_available_actions(state)` care primește o stare și întoarce acțiunile corecte (o listă cu acele coloane care nu sunt _pline_).

Funcția `apply_action(state, action)` este deja implementată (întoarce o stare nouă, nu o modifică pe cea dată ca argument).

In [33]:
# Funcție ce întoarce acțiunile valide dintr-o stare dată
def get_available_actions(state):
    return [action for action in range(WIDTH) if 0 in state[BOARD][action]]

from copy import deepcopy
from functools import reduce

# Funcție ce întoarce starea în care se ajunge prin aplicarea unei acțiuni
def apply_action(state, action):
    if action >= len(state[BOARD]) or 0 not in state[BOARD][action]:
        print("Action " + str(action) + " is not valid.")
        return None
    new_board = deepcopy(state[BOARD])
    new_board[action][new_board[action].index(0,0)] = state[NEXT_PLAYER]
    return (new_board, 3 - state[NEXT_PLAYER])


# Se afișează starea la care se ajunge prin aplicarea unor acțiuni
somestate = reduce(apply_action, [3, 4, 3, 2, 2, 6, 3, 3, 3, 3], init_state())
print_state(somestate)
get_available_actions(somestate) # acțiuni disponibile: [0, 1, 2, 4, 5, 6]

|   A   |
|   R   |
|   A   |
|   R   |
|  RR   |
|  ARA A|
+-------+
Urmează: 1 - ROȘU


[0, 1, 2, 4, 5, 6]

### Stările finale

Pentru a verifica dacă o stare este finală:
 - se verifică dacă ultimul jucător care a mutat a câștigat
 - sau dacă matricea este _plină_
 
Funcția `is_final(state)` va întoarce:
 - `False` dacă starea nu este finală,
 - `1` dacă a câștigat Roșu,
 - `2` dacă a câștigat Albastru și
 - `3` dacă este remiză.

In [34]:
# Funcție ce verifică dacă o stare este finală
def is_final(state):
    # Verificăm dacă matricea este plină
    if not any([0 in col for col in state[BOARD]]): return 3
    # Jucătorul care doar ce a mutat ar putea să fie câștigător
    player = 3 - state[NEXT_PLAYER]
    
    ok = lambda pos: all([state[BOARD][c][r] == player for (r, c) in pos])
    # Verificăm orizontale
    for row in range(HEIGHT):
        for col in range(WIDTH - 3):
            if ok([(row, col + i) for i in range(4)]): return player
    # Verificăm verticale
    for col in range(WIDTH):
        for row in range(HEIGHT - 3):
            if ok([(row + i, col) for i in range(4)]): return player
    # Verificăm diagonale
    for col in range(WIDTH - 3):
        for row in range(HEIGHT - 3):
            if ok([(row + i, col + i) for i in range(4)]): return player
    for col in range(WIDTH - 3):
        for row in range(HEIGHT - 3):
            if ok([(row + i, col + 3 - i) for i in range(4)]): return player
    return False

In [35]:
# Ajungem la o stare finală oarecare și o afișăm.
from random import choice

rand_state = init_state()
print(rand_state)
while not is_final(rand_state):
    actions = get_available_actions(rand_state)
    if not actions:
        break
    action = choice(get_available_actions(rand_state))
    rand_state = apply_action(rand_state, action)

print_state(rand_state)
print("Învingător: %s" % (name[is_final(rand_state)]))

([[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 1)
|R   RA |
|AA  RR |
|AR AAA |
|RRARRA |
|RARARRA|
|ARRRAAA|
+-------+
Urmează: 1 - ROȘU
Învingător: ALBASTRU


In [36]:
# Exemplu: Se afișează starea obținută prin aplicarea unor acțiuni
all_actions = [1, 2, 1, 3, 1, 4, 2, 5]
some_state = reduce(apply_action, all_actions, init_state())
print_state(some_state)
print("Învingător: %s" % (name[is_final(some_state)]))

|       |
|       |
|       |
| R     |
| RR    |
| RAAAA |
+-------+
Urmează: 1 - ROȘU
Învingător: ALBASTRU


## Algoritmul UCT

Algoritmii din familia MCTS conțin patru etape importante:
 - **selecție** - o strategie de alegere a unei acțiuni pentru a exploata
 - **extindere** - construirea unui nod nou în arbore
 - **simulare** - desfășurarea unui joc în mod aleator către o stare finală
 - **propagare înapoi** - actualizarea statisticilor pentru toate nodurile


### Reprezentarea unui nod

Un nod din arborele de stări va fi un dicționar ce conține:
 - numărul de vizitări `N` -- de câte ori s-au realizat simulări din acel nod sau dintr-un descendent al său.
 - valoarea estimată `Q` -- o indicație a calității nodului, bazat pe numărul jocurilor câștigate pornind din acel nod.
 - o referință la starea care îi corespunde -- `STATE`
 - o referință către nodul părinte -- `PARENT`
 - lista copiilor -- `ACTIONS` -- un dicționar care conține, pentru fiecare acțiune explorată, o referință către nodul următor

Exemplu de nod corespunzător unei stări.

    {N: 7, Q: 2.5, STATE: ([[0, 0, 0, 0, 0, 0], ...], 1), PARENT: None, ACTIONS: {0: {N: 3, ...}, 1: {N: 4, ...}}}
    
### Desfășurarea algoritmului

1. Dacă algoritmul pornește cu un arbore gol (fără memorie), atunci se construiește un nod nou.
   Altfel se alege subarborele corespunzător ultimei acțiuni a adversarului. (`TODO3`)

2. Până când se atinge limita bugetului de calcul:
  1. pornind din rădăcină, se alege succesiv un nod următor până când se atinge o stare finală (`is_final`) sau un nod din care nu s-au explorat toate acțiunile posibile (`TODO2` și `TODO4`)
  2. pentru un nod care nu este final și din care nu s-au explorat toate acțiunile, se construiește un nod-copil pentru una dintre acțiunile neexplorate
  3. se simulează un joc pornind din nodul nou până într-o stare finală (`TODO5`)
  4. se evaluează starea finală și se calculează o recompensă (`TODO6`)
  5. se propagă înapoi acea recompensă, actualizându-se și statisticile (numărul de vizite) pentru fiecare nod până la rădăcină (`TODO7`)

In [37]:
# Constante

N = 'N'
Q = 'Q'
STATE = 'state'
PARENT = 'parent'
ACTIONS = 'actions'

### Afișarea unui arbore

In [38]:
def print_tree(tree, indent = 0):
    if not tree:
        return
    tab = "".join(" " * indent)
    print("%sN = %d, Q = %f" % (tab, tree[N], tree[Q]))
    for a in tree[ACTIONS]:
        print("%s %d ==> " % (tab, a))
        print_tree(tree[ACTIONS][a], indent + 3)

### Algoritm

In [39]:
# Funcție ce întoarce un nod nou,
# eventual copilul unui nod dat ca argument
def init_node(state, parent = None):
    return {N: 0, Q: 0, STATE: state, PARENT: parent, ACTIONS: {}}

Selecția se face pe un node care are toate acțiunile explorate, și se alege acțiunea $a$ care maximizează

$ \frac{Q_a}{N_a} + c \cdot \sqrt{\frac{2 \cdot \log{N_{node}}}{N_a}} $

unde $N_a$ și $Q_a$ corespund copilului corespunzător acțiunii $a$, iar $N_{node}$ corespunde nodului pentru care facem selecția (parametrul `node`).

In [40]:
from math import sqrt, log

# constanta care reglează raportul între explorare și exploatare (CP = 0 -> doar exploatare)
CP = 1.0 / sqrt(2.0)

# Funcție ce alege o acțiune dintr-un nod
def select_action(node, c = CP):
    N_node = node[N]

    # <2>
    # Se caută acțiunea a care maximizează expresia:
    # Q_a / N_a  +  c * sqrt(2 * log(N_node) / N_a)
    
    best_score = -1
    best_action = None

    for action in node[ACTIONS]:

        child = node[ACTIONS][action]
        score = child[Q] / child[N] + c * sqrt(2 * log(N_node) / child[N])

        if score > best_score:
            best_score = score
            best_action = action

    return best_action

# Scurtă testare; stările nu sunt relevante aici
test_root = {N: 6, Q: 0.75, STATE: None, PARENT: None, ACTIONS: {}}
test_root[ACTIONS][3] = {N: 4, Q: 0.9, STATE: None, PARENT: test_root, ACTIONS: {}}
test_root[ACTIONS][5] = {N: 2, Q: 0.1, STATE: None, PARENT: test_root, ACTIONS: {}}

print(select_action(test_root, CP))  # ==> 5 (0.8942 < 0.9965)
print(select_action(test_root, 0.3)) # ==> 3 (0.50895 > 0.45157)

5
3


In [41]:
# Algoritmul MCTS (UCT)
#  state0 - starea pentru care trebuie aleasă o acțiune
#  budget - numărul de iterații permis / numărul de noduri care va fi construit
#  tree - un arbore din explorările anterioare, dacă există
#  opponent_s_action - ultima acțiune a adversarului, dacă există

def mcts(state0, budget, tree, opponent_s_action = None):
    # <3>
    # DACĂ există un arbore construit anterior ȘI
    #   acesta are un copil ce corespunde ultimei acțiuni a adversarului,
    # ATUNCI acel copil va deveni nodul de început pentru algoritm.
    # ALTFEL, arborele de start este un nod gol.

    if tree and opponent_s_action in tree[ACTIONS]:
        tree = tree[ACTIONS][opponent_s_action]
    else:
        tree = init_node(state0)
    
    #---------------------------------------------------------------

    for x in range(budget):
        # Pornim procesul de selecție din nodul rădăcină / starea inițială
        node = tree
        state = state0

        # <4>
        # Coborâm în arbore până când ajungem la o stare finală
        # sau la un nod cu acțiuni neexplorate.

        while (not is_final(state) and all(action in node[ACTIONS] for action in get_available_actions(state))):
            new_action = select_action(node)
            state = apply_action(state, new_action)
            node = node[ACTIONS][new_action]
        
        #---------------------------------------------------------------
        
        # <5>
        # Dacă am ajuns într-un nod care nu este final și din care nu s-au
        # `încercat` toate acțiunile, construim un nod nou.
            
        if not is_final(node[STATE]):
            new_action = choice([action for action in get_available_actions(node[STATE]) if action not in node[ACTIONS]])
            state = apply_action(state, new_action)
            
            node = init_node(state, node)
            node[PARENT][ACTIONS][new_action] = node
            
        #---------------------------------------------------------------
        
        # <6>
        # Se simulează o desfășurare a jocului până la ajungerea într-o
        # starea finală. Se evaluează recompensa în acea stare.
            
        state = node[STATE] # de înlocuit cu node[STATE]

        while not is_final(state):
            state = apply_action(state, choice(get_available_actions(state)))
        
        winner = is_final(state)
        if winner == state0[NEXT_PLAYER]:
            reward = 1
        elif winner == (3 - state0[NEXT_PLAYER]):
            reward = 0.0
        elif winner == 3:
            reward = 0.25
        else:
            reward = 0.5
        #---------------------------------------------------------------

        # <7>
        # Se actualizează toate nodurile de la node către rădăcină:
        #  - se incrementează valoarea N din fiecare nod
        #  - pentru nodurile corespunzătoare acestui jucător, se adună recompensa la valoarea Q
        #  - pentru nodurile celelalte, valoarea Q se adună 1 cu minus recompensa
            
        while node:
            node[N] += 1

            if node[STATE][NEXT_PLAYER] == state0[NEXT_PLAYER]:
                node[Q] += reward
            else:
                node[Q] += 1 - reward
                
            node = node[PARENT]
        
        #---------------------------------------------------------------

    if tree:
        final_action = select_action(tree, 0.0)
        return (final_action, tree[ACTIONS][final_action])
    # Acest cod este aici doar ca să nu dea erori testele mai jos; în mod normal tree nu trebuie să fie None
    if get_available_actions(state0):
        return (get_available_actions(state0)[0], init_node())
    return (0, init_node(state0))

In [42]:
# Testare MCTS
(action, tree) = mcts(init_state(), 20, None, None)
print(action)
if tree: print_tree(tree[PARENT]) # Trebuie ca arborele să fie destul de echilibrat, nu dezechilibrat

0
N = 20, Q = 11.000000
 6 ==> 
   N = 3, Q = 1.000000
    4 ==> 
      N = 1, Q = 0.000000
    1 ==> 
      N = 1, Q = 1.000000
 5 ==> 
   N = 3, Q = 2.000000
    5 ==> 
      N = 1, Q = 1.000000
    1 ==> 
      N = 1, Q = 0.000000
 0 ==> 
   N = 5, Q = 4.000000
    2 ==> 
      N = 1, Q = 0.000000
    5 ==> 
      N = 1, Q = 0.000000
    4 ==> 
      N = 1, Q = 0.000000
    3 ==> 
      N = 1, Q = 1.000000
 4 ==> 
   N = 3, Q = 1.000000
    4 ==> 
      N = 1, Q = 1.000000
    5 ==> 
      N = 1, Q = 1.000000
 3 ==> 
   N = 2, Q = 0.000000
    4 ==> 
      N = 1, Q = 1.000000
 1 ==> 
   N = 2, Q = 1.000000
    1 ==> 
      N = 1, Q = 0.000000
 2 ==> 
   N = 2, Q = 0.000000
    0 ==> 
      N = 1, Q = 1.000000


## Evaluarea algoritmului

Funcția de mai jos opune doi jucători ce folosesc algoritmul UCT pentru a decide asupra acțiunii dintr-o stare.

In [43]:
def play_games(games_no, budget1, budget2, verbose = False):
    # Efortul de căutare al jucătorilor
    budget = [budget1, budget2]
    
    score = {p: 0 for p in name}
        
    for i in range(games_no):
        # Memoriile inițiale
        memory = [None, None]
        
        # Se desfășoară jocul
        state = init_state()
        last_action = None
    
        while state and not is_final(state):
            p = state[NEXT_PLAYER] - 1
            (action, memory[p]) = mcts(state, budget[p], memory[p], last_action)
            state = apply_action(state, action)
            last_action = action
        
        # Cine a câștigat?
        if(state):
            winner = is_final(state)
            score[name[winner]] += + 1
        
        # Afișăm
        if verbose and state:
            print_state(state)
            if winner == 3: print("Remiză.")
            else: print("A câștigat %s" % name[winner])

    # Afișează scorul final
    if verbose:
        print("Scor final: %s." % (str(score)))
    return score

In [44]:
# play_games(N, BR, BA, VERBOSE) - rulează N jocuri, cu bugetele BR pt ROȘU și BA pt ALBASTRU
# TODO: rulați pentru câte 5 jocuri:
play_games(5, 2, 30, True) # ne așteptăm să câștige ALBASTRU
play_games(5, 30, 2, True) # ne așteptăm să câștige ROȘU

# TODO: rulați pentru câte 20 de jocuri:
ngames = 20
print(f"Rezultate pentru câte {ngames} de jocuri (ROȘU / ALBASTRU / REMIZĂ):")
print("Buget mic | Buget mare | avantaj ALBASTRU | avantaj ROȘU")
for small, big in [(2, 30), (5, 30), (10, 30), (20, 20)]:
    print(f"{small : >5}     | {big : >6}     |", end = "")
    score = play_games(ngames, small, big, False)
    print("{:>16}".format(f"{score[name[RED]]} / {score[name[BLUE]]} / {score[name[DRAW]]}"), end = "  |")
    score = play_games(ngames, big, small, False)
    print("{:>12}".format(f"{score[name[RED]]} / {score[name[BLUE]]} / {score[name[DRAW]]}"), end = "  |")
    print()
    

|AAA RRR|
|RRR RAA|
|AAA ARR|
|RAA ARA|
|AAA RAR|
|RRRRARR|
+-------+
Urmează: 2 - ALBASTRU
A câștigat ROȘU
|      A|
|A  R  A|
|R  RRRR|
|AR AARR|
|ARARARA|
|ARRARAA|
+-------+
Urmează: 2 - ALBASTRU
A câștigat ROȘU
|       |
|       |
| R     |
|ARARRA |
|ARRRAA |
|RRRAAA |
+-------+
Urmează: 2 - ALBASTRU
A câștigat ROȘU
|  A R  |
|A R ARA|
|ARR RAA|
|ARA ARR|
|RRA RRA|
|ARR RAA|
+-------+
Urmează: 2 - ALBASTRU
A câștigat ROȘU
|AA  R  |
|AA  R  |
|RR  R A|
|RARARRA|
|ARRAARR|
|AARRAAR|
+-------+
Urmează: 2 - ALBASTRU
A câștigat ROȘU
Scor final: {'': 0, 'ROȘU': 5, 'ALBASTRU': 0, 'REMIZĂ': 0}.
|    R  |
|    A  |
| A  A  |
|RAA R  |
|RAARARR|
|RAARRAR|
+-------+
Urmează: 1 - ROȘU
A câștigat ALBASTRU
|RR A A |
|AR A A |
|RA A R |
|RR RARR|
|ARAARAA|
|ARARRRA|
+-------+
Urmează: 1 - ROȘU
A câștigat ALBASTRU
|A   R R|
|R   A R|
|AA  A A|
|RAR AAR|
|RARRRAR|
|AARRRAA|
+-------+
Urmează: 1 - ROȘU
A câștigat ALBASTRU
|       |
|       |
|    A  |
| A  A  |
| A  ARR|
| RR ARR|
+-------+
Urmeaz